In [34]:
!pip install planet
import planet
!planet --version

planet, version 2.1.1


In [36]:
! planet auth init


Email: fzhao@mail.ccnu.edu.cn
Password: 
Initialized
Warning - Environment variable PL_API_KEY already exists. To update, with the new value, use the following:
export PL_API_KEY=$(planet auth value)


In [37]:
!planet auth value

ca411395dfe849aeae7877c7341e0d4f


In [41]:
!export PL_API_KEY=ca411395dfe849aeae7877c7341e0d4f

In [42]:
!echo $PL_API_KEY

ca411395dfe849aeae7877c7341e0d4f


In [43]:
import json
datastring = r'/content/final_studyarea.json'
# import json

# 用json库读取数据
with open(datastring, encoding="utf8") as f:
    data = json.loads(f.read())
# print(data)
geojson_dict = dict(data)
coordinates = geojson_dict['features'][0]['geometry']['coordinates']
print(coordinates)

[[[111.99403559761487, 32.21448596686321], [111.99403559761487, 32.414150144402925], [112.21046747490698, 32.414150144402925], [112.21046747490698, 32.21448596686321], [111.99403559761487, 32.21448596686321]]]


In [78]:
geojson_geometry = {
  "type": "Polygon",
  "coordinates": coordinates
}
# get images that overlap with our AOI
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2022-05-30T00:00:00.000Z",
    "lte": "2022-07-30T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.6
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [79]:
import os
import json
import requests
from requests.auth import HTTPBasicAuth
# os.environ['PL_API_KEY']='ca411395dfe849aeae7877c7341e0d4f'
# API Key stored as an env variable
PLANET_API_KEY = os.getenv('PL_API_KEY')


item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type],
  "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)

geojson = search_result.json()

# let's look at the first result
print(list(geojson.items())[1][1])

[{'_links': {'_self': 'https://api.planet.com/data/v1/item-types/PSScene/items/20220606_024841_21_2446', 'assets': 'https://api.planet.com/data/v1/item-types/PSScene/items/20220606_024841_21_2446/assets/', 'thumbnail': 'https://tiles.planet.com/data/v1/item-types/PSScene/items/20220606_024841_21_2446/thumb'}, '_permissions': ['assets.basic_analytic_4b:download', 'assets.basic_analytic_4b_rpc:download', 'assets.basic_analytic_4b_xml:download', 'assets.basic_analytic_8b:download', 'assets.basic_analytic_8b_xml:download', 'assets.basic_udm2:download', 'assets.ortho_analytic_4b:download', 'assets.ortho_analytic_4b_sr:download', 'assets.ortho_analytic_4b_xml:download', 'assets.ortho_analytic_8b:download', 'assets.ortho_analytic_8b_sr:download', 'assets.ortho_analytic_8b_xml:download', 'assets.ortho_udm2:download', 'assets.ortho_visual:download'], 'assets': ['basic_analytic_4b', 'basic_analytic_4b_rpc', 'basic_analytic_4b_xml', 'basic_analytic_8b', 'basic_analytic_8b_xml', 'basic_udm2', 'ort

In [80]:
image_ids = [feature['id'] for feature in geojson['features']]
print(len(image_ids))
print(image_ids)

174
['20220606_024841_21_2446', '20220606_024838_93_2446', '20220729_021536_84_2465', '20220729_021539_13_2465', '20220729_030507_06_2274', '20220729_030509_35_2274', '20220729_030511_65_2274', '20220726_024808_02_2499', '20220726_025050_62_249b', '20220726_024803_45_2499', '20220726_024805_74_2499', '20220725_024900_11_248c', '20220724_024949_53_247b', '20220724_024947_26_247b', '20220724_024951_81_247b', '20220724_021649_75_2423', '20220723_021818_00_2457', '20220723_021820_30_2457', '20220723_021624_20_2421', '20220723_021628_79_2421', '20220723_021626_50_2421', '20220723_023357_44_2262', '20220723_023352_88_2262', '20220723_023355_16_2262', '20220721_030403_05_2254', '20220721_030405_33_2254', '20220721_025132_39_248f', '20220719_021336_97_2455', '20220719_021842_88_2429', '20220718_030554_98_2414', '20220718_021351_45_2460', '20220718_030559_55_2414', '20220718_030557_26_2414', '20220718_021353_74_2460', '20220717_024735_58_24a4', '20220717_024733_30_24a4', '20220717_024737_86_24a

In [81]:
# Google Earth Engine configuration
cloud_config = planet.order_request.google_earth_engine(
    project='ee-my-zhanghan5016', collection='planetdata')
# Order delivery configuration
delivery_config = planet.order_request.delivery(cloud_config=cloud_config)

In [82]:
iowa_images = image_ids
# Product description for the order request
data_products = [
    planet.order_request.product(item_ids=iowa_images,
                                 product_bundle='analytic_sr_udm2',
                                 item_type='PSScene')
]

# Build the order request
iowa_order = planet.order_request.build_request(name='iowa_order',
                                                products=data_products,
                                                delivery=delivery_config)

print(iowa_order)

{'name': 'iowa_order', 'products': [{'item_ids': ['20220606_024841_21_2446', '20220606_024838_93_2446', '20220729_021536_84_2465', '20220729_021539_13_2465', '20220729_030507_06_2274', '20220729_030509_35_2274', '20220729_030511_65_2274', '20220726_024808_02_2499', '20220726_025050_62_249b', '20220726_024803_45_2499', '20220726_024805_74_2499', '20220725_024900_11_248c', '20220724_024949_53_247b', '20220724_024947_26_247b', '20220724_024951_81_247b', '20220724_021649_75_2423', '20220723_021818_00_2457', '20220723_021820_30_2457', '20220723_021624_20_2421', '20220723_021628_79_2421', '20220723_021626_50_2421', '20220723_023357_44_2262', '20220723_023352_88_2262', '20220723_023355_16_2262', '20220721_030403_05_2254', '20220721_030405_33_2254', '20220721_025132_39_248f', '20220719_021336_97_2455', '20220719_021842_88_2429', '20220718_030554_98_2414', '20220718_021351_45_2460', '20220718_030559_55_2414', '20220718_030557_26_2414', '20220718_021353_74_2460', '20220717_024735_58_24a4', '2022

In [83]:
iowa_aoi =  geojson_geometry
# Clip images to the AOI's perimeter and harmonize the data with Dove Classic
tools = [
    planet.order_request.clip_tool(iowa_aoi),
    planet.order_request.harmonize_tool('Sentinel-2')
]

# Build the order request
iowa_order = planet.order_request.build_request(name='iowa_order',
                                                products=data_products,
                                                delivery=delivery_config,
                                                tools=tools)

print(iowa_order)

{'name': 'iowa_order', 'products': [{'item_ids': ['20220606_024841_21_2446', '20220606_024838_93_2446', '20220729_021536_84_2465', '20220729_021539_13_2465', '20220729_030507_06_2274', '20220729_030509_35_2274', '20220729_030511_65_2274', '20220726_024808_02_2499', '20220726_025050_62_249b', '20220726_024803_45_2499', '20220726_024805_74_2499', '20220725_024900_11_248c', '20220724_024949_53_247b', '20220724_024947_26_247b', '20220724_024951_81_247b', '20220724_021649_75_2423', '20220723_021818_00_2457', '20220723_021820_30_2457', '20220723_021624_20_2421', '20220723_021628_79_2421', '20220723_021626_50_2421', '20220723_023357_44_2262', '20220723_023352_88_2262', '20220723_023355_16_2262', '20220721_030403_05_2254', '20220721_030405_33_2254', '20220721_025132_39_248f', '20220719_021336_97_2455', '20220719_021842_88_2429', '20220718_030554_98_2414', '20220718_021351_45_2460', '20220718_030559_55_2414', '20220718_030557_26_2414', '20220718_021353_74_2460', '20220717_024735_58_24a4', '2022

In [84]:
async def create_and_deliver_order(order_request, client):
    '''Create and deliver an order.

    Parameters:
        order_request: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_request)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'],
                          callback=reporter.update_state,
                          max_attempts=0)

    # Grab the details of the orders
    order_details = await client.get_order(order_id=order['id'])

    return order_details
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GEE
    order_details = await create_and_deliver_order(iowa_order, client)

47:20 - order 3ab31322-783d-4707-bf28-24c93084afaf - state: success


In [105]:
async def create_and_deliver_order(order_request, client):
    '''Create and deliver an order.

    Parameters:
        order_request: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_request)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'],
                          callback=reporter.update_state,
                          max_attempts=0)

    # Grab the details of the orders
    order_details = await client.get_order(order_id=order['id'])

    return order_details
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GEE
    order_details = await create_and_deliver_order(iowa_order, client)

18:56 - order 9effa10f-d3c9-4520-89f1-036e4d97d53d - state: success
